In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

data = pd.read_csv('C:/Users/kname/Desktop/Data/cleaned-odisha-data.csv')

features = ['SO2', 'NO2', 'RSPM/PM10', 'Tempmax', 'Tempmin', 'VeryDenseForest', 
            'ModDenseForest', 'OpenForest', 'TotalForest', 'PercentForest', 
            'Scrub', 'AverageSoilMoisture', 'Population', 'WaterSpread', 
            'MaxDepth', 'StorageCapacity']
target = 'PercentForest'  
missing_cols = [col for col in features if col not in data.columns]
if missing_cols:
    print(f"The following columns are missing from the data: {missing_cols}")
    features = [col for col in features if col in data.columns]

X = data[features]
y = data[target]

imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

green_scores = rf_model.predict(X_scaled)

green_scores_normalized = (green_scores - np.min(green_scores)) / (np.max(green_scores) - np.min(green_scores))

def get_priority(score):
    if score < 0.33:
        return "Low Priority"
    elif score < 0.67:
        return "Moderate Priority"
    else:
        return "High Priority"

results = pd.DataFrame({
    'District': data['District'],
    'Green_Score': green_scores_normalized,
    'Priority': [get_priority(score) for score in green_scores_normalized]
})

results.to_csv('green_scores.csv', index=False)

print("Green scores have been generated and saved to 'green_scores.csv'")


The following columns are missing from the data: ['Tempmax', 'Tempmin']
Green scores have been generated and saved to 'green_scores.csv'
